# All three parts of the assignment are in this notebook together. They start with markdown cells "Part 1", "Part 2", "Part 3".

# Part 1

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [3]:
# download data from wikipedia
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050'

df = pd.read_html(url)

# create new dataframe with the three required columns
df_postcode = df[0]

print(df_postcode.shape)

df_postcode.head()

(287, 3)


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Data cleaning.

In [4]:
df_postcode = df_postcode[df_postcode['Borough'] != 'Not assigned']
print(df_postcode.shape)
df_postcode.head()

(210, 3)


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [5]:
p, b, n = [], [], []
for postcode, borough, neigh in zip(df_postcode['Postcode'], df_postcode['Borough'], df_postcode['Neighbourhood']):
    p.append(postcode)
    b.append(borough)
    if neigh == 'Not assigned':
        n.append(borough)
    else:
        n.append(neigh)

df_postcode = pd.DataFrame({'Postcode': p, 'Borough': b, 'Neighbourhood':n})
print(df_postcode.shape)
df_postcode.head()

(210, 3)


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [6]:
df = df_postcode.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(list)
df = df.reset_index()
df['Neighbourhood'] = df['Neighbourhood'].str.join(', ')
print(df.shape)
df.head()

(103, 3)


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


The list has 103 rows, which means there are 103 different postcodes.

# Part 2

Use geocoder to obtain geo-coordinates for each neighborhood.

In [7]:
import geocoder

lat_list = []
lng_list = []

for i in range(df.shape[0]):
    address = '{}, Toronto, Ontario'.format(df.at[i,'Postcode'])
    g = geocoder.arcgis(address)
    lat_list.append(g.latlng[0])
    lng_list.append(g.latlng[1])


In [8]:
df['Latitude'] = lat_list
df['Longitude'] = lng_list
df.head(20)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.81139,-79.19662
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.78574,-79.15875
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.76575,-79.17470
3,M1G,Scarborough,Woburn,43.76812,-79.21761
4,M1H,Scarborough,Cedarbrae,43.76944,-79.23892
5,M1J,Scarborough,Scarborough Village,43.74446,-79.23117
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.72582,-79.26461
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.71289,-79.28506
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.72360,-79.23496
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.69510,-79.26466


# Part 3

#### Use geopy library to get the latitude and longitude values of New York City.



In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>trt_explorer</em>, as shown below.



In [109]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="trt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Ontario are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Ontario are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top.



In [119]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

For illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in Scarborough. So let's slice the original dataframe and create a new dataframe of the Scarborough data.



In [117]:
scar_data = df[df['Borough'] == 'Scarborough'].reset_index(drop=True)
scar_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.81139,-79.19662
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.78574,-79.15875
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.76575,-79.17470
3,M1G,Scarborough,Woburn,43.76812,-79.21761
4,M1H,Scarborough,Cedarbrae,43.76944,-79.23892


Let's get the geographical coordinates of Scarborough.


In [118]:
address = 'Scarborough, Toronto, Ontario'

geolocator = Nominatim(user_agent="trt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough, Toronto, Ontario are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough, Toronto, Ontario are 43.7729744, -79.2576479.


In [124]:
# create map of Scarborough using latitude and longitude values
map_scar = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(scar_data['Latitude'], scar_data['Longitude'], scar_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scar)  
    
map_scar

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.


#### Define Foursquare Credentials and Version



In [125]:
CLIENT_ID = 'CLEQTPEJZ2OIHXDZ3UIP3TRWXQY4BZPWWMIWL0OTMQP4QGH5' # your Foursquare ID
CLIENT_SECRET = 'WUSTFQIKWOQVKKNZY4OJTDU2NAIHRV0HIOHQPNLSBZZSV2G2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CLEQTPEJZ2OIHXDZ3UIP3TRWXQY4BZPWWMIWL0OTMQP4QGH5
CLIENT_SECRET:WUSTFQIKWOQVKKNZY4OJTDU2NAIHRV0HIOHQPNLSBZZSV2G2


#### Let's explore the first neighborhood in our dataframe.


In [126]:
scar_data.loc[0, 'Neighbourhood']

'Rouge, Malvern'

In [127]:
neighbourhood_latitude = scar_data.loc[0, 'Latitude'] # neighbourhood latitude value
neighbourhood_longitude = scar_data.loc[0, 'Longitude'] # neighbourhood longitude value

neighbourhood_name = scar_data.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Rouge, Malvern are 43.811390000000074, -79.19661999999994.


#### Now, let's get the top 100 venues within a radius of 500 meters.


In [128]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=CLEQTPEJZ2OIHXDZ3UIP3TRWXQY4BZPWWMIWL0OTMQP4QGH5&client_secret=WUSTFQIKWOQVKKNZY4OJTDU2NAIHRV0HIOHQPNLSBZZSV2G2&v=20180605&ll=43.811390000000074,-79.19661999999994&radius=500&limit=100'

In [129]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '605ead76cce03f6f704d513e'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.815890004500076,
    'lng': -79.19039569057789},
   'sw': {'lat': 43.80688999550007, 'lng': -79.20284430942199}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': 'Wendy’s',
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label':

From the Foursquare lab in the previous module, we know that all the information is in the _items_ key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.


In [130]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [133]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

nearby_venues.head()

2 venues were returned by Foursquare.


<ipython-input-133-5d88366b15f3>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Wendy’s,Fast Food Restaurant,43.807448,-79.199056
1,Shyne Display Group,Furniture / Home Store,43.809040,-79.199596


## Explore Neighbourhoods in Scarborough



In [134]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [176]:
scar_venues = getNearbyVenues(names=scar_data['Neighbourhood'],
                                   latitudes=scar_data['Latitude'],
                                   longitudes=scar_data['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge


In [178]:
print(scar_venues.shape)
scar_venues.head()

(90, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.81139,-79.19662,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge, Malvern",43.81139,-79.19662,Shyne Display Group,43.809040,-79.199596,Furniture / Home Store
2,"Highland Creek, Rouge Hill, Port Union",43.78574,-79.15875,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
3,"Highland Creek, Rouge Hill, Port Union",43.78574,-79.15875,Royal Canadian Legion,43.782533,-79.163085,Bar
4,"Guildwood, Morningside, West Hill",43.76575,-79.17470,Homestead Roofing Repair,43.765140,-79.178663,Construction & Landscaping


In [179]:
scar_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,12,12,12,12,12,12
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,2,2,2,2,2,2
"Clairlea, Golden Mile, Oakridge",9,9,9,9,9,9
"Clarks Corners, Sullivan, Tam O'Shanter",12,12,12,12,12,12
"Cliffcrest, Cliffside, Scarborough Village West",8,8,8,8,8,8
"Dorset Park, Scarborough Town Centre, Wexford Heights",4,4,4,4,4,4
"East Birchmount Park, Ionview, Kennedy Park",5,5,5,5,5,5


In [180]:
print('There are {} uniques categories.'.format(len(scar_venues['Venue Category'].unique())))

There are 48 uniques categories.


## Analyze Each Neighborhood



In [181]:
# one hot encoding
scar_onehot = pd.get_dummies(scar_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
scar_onehot['Neighbourhood'] = scar_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [scar_onehot.columns[-1]] + list(scar_onehot.columns[:-1])
scar_onehot = scar_onehot[fixed_columns]

scar_onehot.head()

,Neighbourhood,Auto Garage,Badminton Court,Bakery,Bank,Bar,Bubble Tea Shop,Burger Joint,Bus Line,Business Service,Café,Cantonese Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Deli / Bodega,Department Store,Discount Store,Fast Food Restaurant,Furniture / Home Store,Gaming Cafe,General Entertainment,Gift Shop,Grocery Store,Gym / Fitness Center,Gym Pool,Hobby Shop,Hong Kong Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Korean BBQ Restaurant,Light Rail Station,Metro Station,Newsagent,Other Great Outdoors,Park,Pharmacy,Pizza Place,Restaurant,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Spa,Supermarket,Sushi Restaurant,Trail
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [182]:
scar_onehot.shape

(90, 49)

In [183]:
scar_grouped = scar_onehot.groupby('Neighbourhood').mean().reset_index()
scar_grouped

,Neighbourhood,Auto Garage,Badminton Court,Bakery,Bank,Bar,Bubble Tea Shop,Burger Joint,Bus Line,Business Service,Café,Cantonese Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Deli / Bodega,Department Store,Discount Store,Fast Food Restaurant,Furniture / Home Store,Gaming Cafe,General Entertainment,Gift Shop,Grocery Store,Gym / Fitness Center,Gym Pool,Hobby Shop,Hong Kong Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Korean BBQ Restaurant,Light Rail Station,Metro Station,Newsagent,Other Great Outdoors,Park,Pharmacy,Pizza Place,Restaurant,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Spa,Supermarket,Sushi Restaurant,Trail
0,Agincourt,0.0,0.083333,0.083333,0.000000,0.0,0.083333,0.000000,0.000000,0.0,0.00,0.000000,0.166667,0.000000,0.00,0.000000,0.000000,0.083333,0.083333,0.000000,0.0,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.083333,0.00,0.0,0.000000,0.0,0.00,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.083333,0.00,0.000000,0.0,0.083333,0.083333,0.0
1,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.333333,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.666667,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0
2,"Birch Cliff, Cliffside West",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.25,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.25,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.25,0.000000,0.0,0.000000,0.000000,0.0
3,Cedarbrae,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.5,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.5
4,"Clairlea, Golden Mile, Oakridge",0.0,0.000000,0.222222,0.000000,0.0,0.000000,0.000000,0.111111,0.0,0.00,0.000000,0.000000,0.222222,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.222222,0.0,0.00,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.111111,0.0,0.000000,0.000000,0.0
5,"Clarks Corners, Sullivan, Tam O'Shanter",0.0,0.000000,0.083333,0.083333,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.083333,0.000000,0.083333,0.00,0.000000,0.083333,0.000000,0.000000,0.166667,0.0,0.0,0.00,0.00,0.083333,0.000000,0.000000,0.083333,0.000000,0.00,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.083333,0.083333,0.000,0.083333,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0
6,"Cliffcrest, Cliffside, Scarborough Village West",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.125000,0.00,0.000000,0.000000,0.000000,0.125000,0.000000,0.0,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.125000,0.125000,0.125,0.125000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0
7,"Dorset Park, Scarborough Town Centre, Wexford ...",0.0,0.000000,0.500000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0
8,"East Birchmount Park, Ionview, Kennedy Park",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.200000,0.200000,0.00,0.000000,0.000000,

#### Let's confirm the new size



In [184]:
scar_grouped.shape

(16, 49)

In [185]:
num_top_venues = 5

for hood in scar_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = scar_grouped[scar_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                venue  freq
0  Chinese Restaurant  0.17
1       Shopping Mall  0.08
2     Badminton Court  0.08
3    Department Store  0.08
4           Newsagent  0.08


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                  venue  freq
0              Pharmacy  0.67
1          Intersection  0.33
2           Auto Garage  0.00
3  Other Great Outdoors  0.00
4            Hobby Shop  0.00


----Birch Cliff, Cliffside West----
                   venue  freq
0  General Entertainment  0.25
1           Skating Rink  0.25
2                   Café  0.25
3        College Stadium  0.25
4            Auto Garage  0.00


----Cedarbrae----
                  venue  freq
0                 Trail   0.5
1           Gaming Cafe   0.5
2              Gym Pool   0.0
3  Hong Kong Restaurant   0.0
4        Ice Cream Shop   0.0


----Clairlea, Golden Mile, Oakridge----
          venue  freq
0   Coffee Shop  0.22
1        Bakery  0.22
2  Intersection  0.22
3  Soccer Fie

In [186]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [221]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = scar_grouped['Neighbourhood']

for ind in np.arange(scar_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scar_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Newsagent,Supermarket,Badminton Court,Bakery,Sushi Restaurant,Shopping Mall,Bubble Tea Shop,Discount Store,Department Store
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Pharmacy,Intersection,Trail,Chinese Restaurant,Gaming Cafe,Furniture / Home Store,Fast Food Restaurant,Discount Store,Department Store,Deli / Bodega
2,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,College Stadium,Café,Chinese Restaurant,Gaming Cafe,Furniture / Home Store,Fast Food Restaurant,Discount Store,Department Store
3,Cedarbrae,Trail,Gaming Cafe,Sushi Restaurant,General Entertainment,Furniture / Home Store,Fast Food Restaurant,Discount Store,Department Store,Deli / Bodega,Construction & Landscaping
4,"Clairlea, Golden Mile, Oakridge",Bakery,Intersection,Coffee Shop,Metro Station,Soccer Field,Bus Line,Trail,Furniture / Home Store,Fast Food Restaurant,Discount Store


## Cluster Neighborhoods



In [222]:
# set number of clusters
kclusters = 5

scar_grouped_clustering = scar_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scar_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 4, 3, 1, 1], dtype=int32)

In [223]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

scar_merged = scar_data

# merge scar_grouped with scar_data to add latitude/longitude for each neighbourhood
scar_merged = scar_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

scar_merged = scar_merged[scar_merged['Cluster Labels'].notna()]

scar_merged # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.81139,-79.19662,3.0,Furniture / Home Store,Fast Food Restaurant,Trail,Sushi Restaurant,General Entertainment,Gaming Cafe,Discount Store,Department Store,Deli / Bodega,Construction & Landscaping
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.78574,-79.15875,1.0,Bar,Construction & Landscaping,Trail,Coffee Shop,General Entertainment,Gaming Cafe,Furniture / Home Store,Fast Food Restaurant,Discount Store,Department Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.76575,-79.17470,1.0,Gym / Fitness Center,Construction & Landscaping,Park,Trail,Chinese Restaurant,Gaming Cafe,Furniture / Home Store,Fast Food Restaurant,Discount Store,Department Store
3,M1G,Scarborough,Woburn,43.76812,-79.21761,1.0,Coffee Shop,Indian Restaurant,Business Service,Korean BBQ Restaurant,Park,Trail,Furniture / Home Store,Fast Food Restaurant,Discount Store,Department Store
4,M1H,Scarborough,Cedarbrae,43.76944,-79.23892,2.0,Trail,Gaming Cafe,Sushi Restaurant,General Entertainment,Furniture / Home Store,Fast Food Restaurant,Discount Store,Department Store,Deli / Bodega,Construction & Landscaping
5,M1J,Scarborough,Scarborough Village,43.74446,-79.23117,1.0,Grocery Store,Spa,Restaurant,Indian Restaurant,Park,Chinese Restaurant,Furniture / Home Store,Fast Food Restaurant,Discount Store,Department Store
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.72582,-79.26461,1.0,Chinese Restaurant,Hobby Shop,Discount Store,Department Store,Coffee Shop,Gift Shop,Gaming Cafe,Furniture / Home Store,Fast Food Restaurant,Deli / Bodega
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.71289,-79.28506,1.0,Bakery,Intersection,Coffee Shop,Metro Station,Soccer Field,Bus Line,Trail,Furniture / Home Store,Fast Food Restaurant,Discount Store
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.72360,-79.23496,1.0,Ice Cream Shop,Discount Store,Sandwich Place,Restaurant,Pizza Place,Pharmacy,Coffee Shop,Trail,Cantonese Restaurant,Fast Food Restaurant
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.69510,-79.26466,1.0,General Entertainment,Skating Rink,College Stadium,Café,Chinese Restaurant,Gaming Cafe,Furniture / Home Store,Fast Food Restaurant,Discount Store,Department Store


In [224]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scar_merged['Latitude'], scar_merged['Longitude'], scar_merged['Neighbourhood'], scar_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters



#### Cluster 1



In [225]:
scar_merged.loc[scar_merged['Cluster Labels'] == 0, scar_merged.columns[[1] + list(range(5, scar_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,0.0,Pharmacy,Intersection,Trail,Chinese Restaurant,Gaming Cafe,Furniture / Home Store,Fast Food Restaurant,Discount Store,Department Store,Deli / Bodega


#### Cluster 2




In [226]:
scar_merged.loc[scar_merged['Cluster Labels'] == 1, scar_merged.columns[[1] + list(range(5, scar_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,1.0,Bar,Construction & Landscaping,Trail,Coffee Shop,General Entertainment,Gaming Cafe,Furniture / Home Store,Fast Food Restaurant,Discount Store,Department Store
2,Scarborough,1.0,Gym / Fitness Center,Construction & Landscaping,Park,Trail,Chinese Restaurant,Gaming Cafe,Furniture / Home Store,Fast Food Restaurant,Discount Store,Department Store
3,Scarborough,1.0,Coffee Shop,Indian Restaurant,Business Service,Korean BBQ Restaurant,Park,Trail,Furniture / Home Store,Fast Food Restaurant,Discount Store,Department Store
5,Scarborough,1.0,Grocery Store,Spa,Restaurant,Indian Restaurant,Park,Chinese Restaurant,Furniture / Home Store,Fast Food Restaurant,Discount Store,Department Store
6,Scarborough,1.0,Chinese Restaurant,Hobby Shop,Discount Store,Department Store,Coffee Shop,Gift Shop,Gaming Cafe,Furniture / Home Store,Fast Food Restaurant,Deli / Bodega
7,Scarborough,1.0,Bakery,Intersection,Coffee Shop,Metro Station,Soccer Field,Bus Line,Trail,Furniture / Home Store,Fast Food Restaurant,Discount Store
8,Scarborough,1.0,Ice Cream Shop,Discount Store,Sandwich Place,Restaurant,Pizza Place,Pharmacy,Coffee Shop,Trail,Cantonese Restaurant,Fast Food Restaurant
9,Scarborough,1.0,General Entertainment,Skating Rink,College Stadium,Café,Chinese Restaurant,Gaming Cafe,Furniture / Home Store,Fast Food Restaurant,Discount Store,Department Store
10,Scarborough,1.0,Bakery,Light Rail Station,Gift Shop,Trail,Coffee Shop,Gaming Cafe,Furniture / Home Store,Fast Food Restaurant,Discount Store,Department Store
12,Scarborough,1.0,Chinese Restaurant,Newsagent,Supermarket,Badminton Court,Bakery,Sushi Restaurant,Shopping Mall,Bubble Tea Shop,Discount Store,Department Store


#### Cluster 3




In [227]:
scar_merged.loc[scar_merged['Cluster Labels'] == 2, scar_merged.columns[[1] + list(range(5, scar_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Scarborough,2.0,Trail,Gaming Cafe,Sushi Restaurant,General Entertainment,Furniture / Home Store,Fast Food Restaurant,Discount Store,Department Store,Deli / Bodega,Construction & Landscaping


#### Cluster 4




In [228]:
scar_merged.loc[scar_merged['Cluster Labels'] == 3, scar_merged.columns[[1] + list(range(5, scar_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,3.0,Furniture / Home Store,Fast Food Restaurant,Trail,Sushi Restaurant,General Entertainment,Gaming Cafe,Discount Store,Department Store,Deli / Bodega,Construction & Landscaping


#### Cluster 5




In [230]:
scar_merged.loc[scar_merged['Cluster Labels'] == 4, scar_merged.columns[[1] + list(range(5, scar_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Scarborough,4.0,Auto Garage,Sushi Restaurant,General Entertainment,Gaming Cafe,Furniture / Home Store,Fast Food Restaurant,Discount Store,Department Store,Deli / Bodega,Construction & Landscaping
